## Melisa / 05.11.2023
### Data Preprocessing
#### Task: Preprocess ’review text’ Data


In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
"""
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")
"""

'\nnltk.download("stopwords")\nnltk.download("punkt")\nnltk.download("wordnet")\n'

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [3]:
df = pd.read_json("../data/spoiler.json", lines=True)

### DOING THE PREPROCESSING ON A SINGULAR DATA POINT TO MAKE SURE ALL IS WELL.
## TODO: TURN ALL OF THIS INTO A FUNCTION SO YOU CAN USE APPLY FUNCTION TO THE DATAFRAME ON THE MAIN SOURCE FILE.

In [17]:
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def process_review_text(val):
    clean_test = []
    for sentence in val:
        # Tokenize the sentence
        words = word_tokenize(sentence[1])
        # Lowercase and remove non-alphanumeric characters
        words = [word.lower() for word in words if word.isalnum()]
        # Remove stopwords 
        words = [word for word in words if word not in stop_words]
        # Append the cleaned sentence to the cleaned dataset
        
        #LEMMATIZATION ÇOK BİR ŞEY DEĞİŞTİRMİYOR, YETERLİ Mİ YOKSA STEMMİNG Mİ YAPMALIYIZ? İLERDE DURUMA GÖRE BURAYA GERİ DÖN
        normalized_words = [lemmatizer.lemmatize(word) for word in words]
        clean_test.append(normalized_words)
        
    
    # Build a vocabulary
    vocab = set(word for sentence in clean_test for word in sentence)
    word_to_index = {word: index for index, word in enumerate(vocab)}

    # NUMERICAL REPRESENTATION
    numerical_data = [[word_to_index[word] for word in sentence] for sentence in clean_test]
    
    
    # Padding sequences / BAŞLANGIÇ OLARAK 13 SEÇTİM AMA CÜMLE UZUNLUĞU MEANİ 8, OLABİLDİĞİNCE ACCURATE OLSUN DİYE. EĞER HIZDAN DOLAYI
    # SIKINTI YAŞAMAYA BAŞLARSAK BURAYI 8-13 ARASI GİTTİKÇE AŞAĞI ÇEKEBİLİRİZ.
    max_sequence_length = 13
    padded_data = [seq[:max_sequence_length] + [0] * max(0, max_sequence_length - len(seq)) for seq in numerical_data]
    
    
    padded_data = torch.tensor(padded_data)

    #SPOILER LABELS
    label_lst = [i[0] for i in val]
    labels = torch.tensor(label_lst)

    dataset = TensorDataset(padded_data, labels)
    #print(padded_data)    
    return dataset


In [20]:
# WORKING ON A SMALL SUBSET FOR NOW JUST TO MAKE SURE EVERYTHING WORKS
smaller_df = df.iloc[:20000]
#print(smaller_df.shape)

smaller_df["test"] = smaller_df["review_sentences"].apply(process_review_text)
#print(smaller_df["test"].describe)

(20000, 8)
<bound method NDFrame.describe of 0        (([tensor(10), tensor(28), tensor(0), tensor(0...
1        (([tensor(5), tensor(2), tensor(0), tensor(0),...
2        (([tensor(129), tensor(33), tensor(149), tenso...
3        (([tensor(10), tensor(5), tensor(8), tensor(6)...
4        (([tensor(135), tensor(17), tensor(34), tensor...
                               ...                        
19995    (([tensor(28), tensor(0), tensor(34), tensor(1...
19996    (([tensor(28), tensor(14), tensor(11), tensor(...
19997    (([tensor(6), tensor(4), tensor(11), tensor(9)...
19998    (([tensor(12), tensor(21), tensor(0), tensor(0...
19999    (([tensor(0), tensor(8), tensor(0), tensor(0),...
Name: test, Length: 20000, dtype: object>


<ipython-input-20-74252055dc8f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smaller_df["test"] = smaller_df["review_sentences"].apply(process_review_text)
